#  자연어 처리를 위한 NLTK와 KoNLPy 설치하기

In [2]:
import urllib.request
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [1]:
from konlpy.tag import Okt  
from konlpy.tag import Mecab
mecab = Mecab()
#okt = Okt()

## NLTK와 NLTK Data 설치
- 엔엘티케이(NLTK)는 자연어 처리를 위한 파이썬 패키지

In [3]:
# !pip install nltk

In [4]:
import nltk
nltk.__version__

'3.6.2'

In [5]:
# 해당 코드를 실행 후에 NLTK 실습에 필요한 각종 패키지와 코퍼스를 다운로드할 수 있다
#nltk.download()

## KoNLPY 설치
- 코엔엘파이(KoNLPy)는 한국어 자연어 처리를 위한 형태소 분석기 패키지

In [6]:
#!pip install konlpy

In [7]:
import konlpy
konlpy.__version__

'0.5.2'

### 한국어 문장분류기(KSS : Korean Sentence Splitter) 설치

In [8]:
#! pip install kss

In [9]:
import kss

# 텍스트 전처리
- 토큰화 : 자연어 처리에서 크롤링 등으로 얻어낸 코퍼스 데이터가 필요에 맞게 전처리되지 않은 상태라면, 해당 데이터를 사용하고자하는 용도에 맞게 토큰화(tokenization) & 정제(cleaning) & 정규화(normalization)하는 일을 하게된다. 이렇듯 주어진 코퍼스에서 토큰이라 불리는 단위로 나눈 작업을 토큰화 작업이라고 부른다.

# Word2Vec

In [10]:
# mecab - 방법1
# https://sosomemo.tistory.com/30
# Mecab설치
'''bash
cd /tmp
sudo wget https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
sudo tar xvf mecab-0.996-ko-0.9.2.tar.gz

cd /tmp/mecab-0.996-ko-0.9.2
sudo ./configure
sudo make check
sudo make install
'''

'''bash
cd /tmp
wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.1.1-20180720.tar.gz
tar zxvf mecab-ko-dic-2.1.1-20180720.tar.gz

cd /tmp/mecab-ko-dic-2.1.1-20180720
sudo ./autogen.sh
sudo ./configure
sudo make
sudo make install
'''

'''bash
cd /tmp
git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
cd mecab-python-0.996
python3 setup.py build
python3 setup.py install
'''

'bash\ncd /tmp\ngit clone https://bitbucket.org/eunjeon/mecab-python-0.996.git\ncd mecab-python-0.996\npython3 setup.py build\npython3 setup.py install\n'

**Word2Vec의 하이퍼파라미터 값**

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
- window = 컨텍스트 윈도우 크기  
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
- workers = 학습을 위한 프로세스 수  
- sg = 0은 CBOW, 1은 Skip-gram.  

## Ko-Wiki Data 전처리

In [11]:
import pandas as pd

ko_wiki_path = '/repo/course/sem21_01/youtube_summarizer/dataset/preprocessed/'
ko_wiki_txt = open(ko_wiki_path+'processed_wiki_ko.txt', 'r')

In [13]:
ko_wiki_para = ko_wiki_txt.readlines()

In [14]:
len(ko_wiki_para)

311237

In [15]:
ko_wiki_para[0]

'제임스 얼 "지미" 카터 주니어 (, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다. 지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다. 1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다. 취임식을 올리는 카터 1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다. 카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다. Carter Begin, Camp David 1978.gif|섬네일|300px|캠프데이비드에서 사다트와 베긴과 함께카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다. 카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다. 그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건

In [16]:
import kss

para1 = ko_wiki_para[0]

print(kss.split_sentences(para1))

['제임스 얼 "지미" 카터 주니어 (, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.', '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '조지아 공과대학교를 졸업하였다.', '그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.', '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.', '그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.', '대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.', '조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.', '취임식을 올리는 카터 1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.', '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.', 'Carter Begin, Camp David 1978.gif|섬네일|300px|캠프데이비드에서 사다트와 베긴과 함께카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.', '그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다.', '1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다.', '또한 소련과 제2차 전략 무기 제한 협상에 조인했다.', '카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.', '그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 

In [17]:
# ## multi-processing
# points_images = [[points_lst[i], images[i]] for i in range(len(images))]
# with Pool(workers or cpu_count()) as pool:
# result = list(tqdm(pool.imap(
#     func=partial(getFeatureDesriptors_, winSize=winSize, num_bins=num_bins),
#     iterable=points_images), total=len(points_images)))
# pool.close()
# pool.join()
# return result   

In [18]:
stopwords = ['의','은','는','이','가','좀','잘','과','도','을','를','으로','자','에','와','한','하다','합니다', '입니다','습니다']

In [92]:
tokenized_wiki_data = []
for para in tqdm(ko_wiki_para):
    para = kss.split_sentences(para) # split sentence
    for sentence in para:
        #print(f'previous : {sentence}')
        tmp_sen = re.sub('[^가-힣a-z]', ' ', sentence) # 영어 소문자와 한글을 제외한 모든 문자를 제거
        temp_X = mecab.morphs(tmp_sen) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        #print(f'after : {temp_X}')
        tokenized_wiki_data.append(temp_X)

100%|██████████| 311237/311237 [2:51:38<00:00, 30.22it/s]


In [94]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/tokenized_dataset/tokenized_wiki_data.pkl'

# save pickle file
with open(pk_path, 'wb') as f:
    pickle.dump(tokenized_wiki_data, f)

In [19]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/tokenized_dataset/tokenized_wiki_data.pkl'

# load pickle file
with open(pk_path, 'rb') as f:
     tokenized_wiki_data = pickle.load(f)

## 뉴스 기사 데이터 전처리

In [20]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
import os 

os.getcwd()

'/repo/course/sem21_01/youtube_summarizer/jupyter'

In [22]:
import json
import pandas as pd

path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_dataset/train.jsonl'

### bfly train
sent_df = pd.read_json(path, lines=True, encoding="utf-8")
print('shape : {}'.format(sent_df.shape))
sent_df.head()

shape : (260697, 5)


,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [23]:
#from konlpy.tag import Okt  
from konlpy.tag import Mecab
mecab = Mecab()
#okt = Okt()

In [104]:
stopwords = ['은','는','이','가','의','좀','잘','과','도','을','를','으로','자','에','와','한','하다','합니다','이다','입니다','습니다']

In [105]:
import re

In [106]:
tokenized_news_data = []
for news in tqdm(sent_df['article_original']):
    for sentence in news:
        #print(f'previous : {sentence}')
        tmp_sen = re.sub('[^가-힣a-z]', ' ', sentence) # 영어 소문자와 한글을 제외한 모든 문자를 제거
        temp_X = mecab.morphs(tmp_sen) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        #print(f'after : {temp_X}')
        tokenized_news_data.append(temp_X)

100%|██████████| 260697/260697 [08:58<00:00, 483.90it/s]


In [107]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/tokenized_dataset/tokenized_news_data.pkl'

# save pickle file
with open(pk_path, 'wb') as f:
    pickle.dump(tokenized_news_data, f)

In [24]:
import pickle

pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/tokenized_dataset/tokenized_news_data.pkl'

# load pickle file
with open(pk_path, 'rb') as f:
     tokenized_news_data = pickle.load(f)

In [25]:
tokenized_news_data[0]

['지난해',
 '고령화',
 '유례',
 '드문',
 '겨울',
 '한파',
 '등',
 '영향',
 '우리나라',
 '사망자',
 '수',
 '통계',
 '작성',
 '이후',
 '가장',
 '많',
 '았',
 '다']

In [26]:
tokenized_wiki_data[0]

['제임스',
 '얼',
 '지미',
 '카터',
 '주니어',
 '년',
 '월',
 '일',
 '민주당',
 '출신',
 '미국',
 '번',
 '째',
 '대통령',
 '년',
 '년',
 '다']

### import pretrained model (사용 X))
**Pretraining 환경**
~~~
def train_word2vec(corpus_fname, model_fname):
    make_save_path(model_fname)
    corpus = Word2VecCorpus(corpus_fname)
    model = Word2Vec(corpus, size=100, workers=4, sg=1)
    model.save(model_fname)
~~~

In [27]:
len(tokenized_wiki_data)

3965859

In [28]:
len(tokenized_news_data)

3300370

In [29]:
train_w2v = list()

In [30]:
train_w2v.extend(tokenized_wiki_data)
train_w2v.extend(tokenized_news_data)
len(train_w2v)

7266229

### Train New Word2Vec

In [127]:
import os

os.cpu_count()

8

In [31]:
import gensim
# Word2Vec
from gensim.models import Word2Vec

# FastText
from gensim.models import FastText
'''
sentences (iterable of iterables, optional) – The sentences iterable can be simply a list of lists of tokens, but for larger corpora, consider an iterable that streams the sentences directly from disk/network. See BrownCorpus, Text8Corpus or LineSentence in word2vec module for such examples. See also the tutorial on data streaming in Python. If you don’t supply sentences, the model is left uninitialized – use if you plan to initialize it in some other way.

size (int, optional) – Dimensionality of the word vectors.

window (int, optional) – Maximum distance between the current and predicted word within a sentence.

min_count (int, optional) – Ignores all words with total frequency lower than this.

workers (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).

sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
'''

w2v_model = Word2Vec(sentences = train_w2v, size = 768, window = 5, workers = 7, sg = 1)

In [35]:
w2v_path = '/repo/course/sem21_01/youtube_summarizer/src/word_embedding/model/'

w2v_model.save(w2v_path+'w2v_model.model')

### Load Saved Word2Vec Model

In [36]:
from gensim.models import Word2Vec

w2v_model_test = Word2Vec.load(w2v_path+'w2v_model.model')

w2v_model_test

In [37]:
result=w2v_model_test.wv.most_similar("강아지")
print(result)

[('고양이', 0.5076916813850403), ('애완견', 0.49656134843826294), ('애견', 0.4751525819301605), ('반려', 0.46863648295402527), ('애견인', 0.4465317726135254), ('견', 0.44478684663772583), ('펫', 0.43852585554122925), ('동물', 0.4298151135444641), ('키워본', 0.42482659220695496), ('폭스테리어', 0.42475879192352295)]


In [ ]:
from gensim.models import KeyedVectors

finetuned_wv.save("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/finetuned_wv.kv")

In [ ]:
reloaded_word_vectors = KeyedVectors.load("/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/finetuned_wv.kv")
reloaded_word_vectors

In [ ]:
reloaded_word_vectors["안녕"]

In [ ]:
reloaded_word_vectors.get_vector("안녕")